# Finetune on ScienceQA

Let's use LLM Engine to fine-tune Llama-2 on ScienceQA!


# Packages Required

For this demo, we'll be using the `scale-llm-engine` package and `datasets` from Huggingface.


In [1]:
!pip install scale-llm-engine
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00


# Data Preparation

Let's load in the dataset using Huggingface and view the features.


In [2]:
from datasets import load_dataset
from smart_open import smart_open
import pandas as pd

dataset = load_dataset('derek-thomas/ScienceQA')
dataset['train'].features

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/12726 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4241 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4241 [00:00<?, ? examples/s]

{'image': Image(decode=True, id=None),
 'question': Value(dtype='string', id=None),
 'choices': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'answer': Value(dtype='int8', id=None),
 'hint': Value(dtype='string', id=None),
 'task': Value(dtype='string', id=None),
 'grade': Value(dtype='string', id=None),
 'subject': Value(dtype='string', id=None),
 'topic': Value(dtype='string', id=None),
 'category': Value(dtype='string', id=None),
 'skill': Value(dtype='string', id=None),
 'lecture': Value(dtype='string', id=None),
 'solution': Value(dtype='string', id=None)}

Now, let's format the dataset into what's acceptable for LLM Engine - a CSV file with 'prompt' and 'response' columns.


In [ ]:
choice_prefixes = [chr(ord('A') + i) for i in range(26)] # A-Z
def format_options(options, choice_prefixes):
    return ' '.join([f'({c}) {o}' for c, o in zip(choice_prefixes, options)])

def format_prompt(r, choice_prefixes):
    options = format_options(r['choices'], choice_prefixes)
    return f'''Context: {r["hint"]}\nQuestion: {r["question"]}\nOptions:{options}\nAnswer:'''

def format_label(r, choice_prefixes):
    return choice_prefixes[r['answer']]

def convert_dataset(ds):
    prompts = [format_prompt(i, choice_prefixes) for i in ds if i['hint'] != '']
    labels = [format_label(i, choice_prefixes) for i in ds if i['hint'] != '']
    df = pd.DataFrame.from_dict({'prompt': prompts, 'response': labels})
    return df

save_to_s3 = False
df_train = convert_dataset(dataset['train'])
if save_to_s3:
    train_url = 's3://...'
    val_url = 's3://...'
    df_train = convert_dataset(dataset['train'])
    with smart_open(train_url, 'wb') as f:
        df_train.to_csv(f)

    df_val = convert_dataset(dataset['validation'])
    with smart_open(val_url, 'wb') as f:
        df_val.to_csv(f)
else:
    # Gists of the already processed datasets
    train_url = 'https://gist.githubusercontent.com/jihan-yin/43f19a86d35bf22fa3551d2806e478ec/raw/91416c09f09d3fca974f81d1f766dd4cadb29789/scienceqa_train.csv'
    val_url = 'https://gist.githubusercontent.com/jihan-yin/43f19a86d35bf22fa3551d2806e478ec/raw/91416c09f09d3fca974f81d1f766dd4cadb29789/scienceqa_val.csv'

df_train

# Fine-tune

Now, we can fine-tune the model using LLM Engine.


In [5]:
import os
os.environ['SCALE_API_KEY'] = '123'

from llmengine import FineTune

response = FineTune.create(
    model="llama-2-7b",
    training_file=train_url,
    validation_file=val_url,
    hyperparameters={
        'lr':2e-4,
    },
    suffix='science-qa-llama'
)
run_id = response.id

We can sleep until the job completes.


In [8]:
print(run_id)

ft-clirl5kh1usg037ciilg


In [16]:
import time

while True:
    job_status = FineTune.get(run_id).status
    print(job_status)
    if job_status == 'SUCCESS':
        break
    time.sleep(60)

fine_tuned_model = FineTune.get(run_id).fine_tuned_model

BatchJobStatus.SUCCESS


# Inference and Evaluation

Let's evaluate the new fine-tuned model by running inference against it.


In [11]:
import pandas as pd
from llmengine import Completion

# Helper function to get outputs for fine-tuned model with retries
def get_output(prompt: str, num_retry: int = 5):
    for _ in range(num_retry):
        try:
            response = Completion.create(
                model=fine_tuned_model,
                prompt=prompt,
                max_new_tokens=1,
                temperature=0.01
            )
            return response.output.text.strip()
        except Exception as e:
            print(e)
    return ""

# Read the test data
test = pd.read_csv(val_url)

#test["prediction"] = test["prompt"].apply(get_output)
#print(f"Accuracy: {(test['response'] == test['prediction']).mean() * 100:.2f}%")

Accuracy: 80.29%


In [14]:
print(type(test))

<class 'pandas.core.frame.DataFrame'>


In [15]:
# Save the DataFrame to a JSON file
test.to_json('output.json', orient='records', lines=True)


In [19]:
from google.colab import drive
drive.mount('/content/drive')
main_path = "drive/MyDrive/ScienceQA_DATA/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import json
def read_json(json_file_path):
  # Open the file in read mode
  with open(json_file_path, 'r') as file:
    # Load the JSON data from the file
    data = json.load(file)
  return data

# Specify the path to your JSON file
json_file_path = main_path + 'im_captioned_llava_cot_combined_test_result.json'
problem_data = read_json(json_file_path)


In [ ]:
c=0
for pid, content in problem_data.items():
  #print(problem_data[pid]['image_captioned_prompt'])
  print(pid)
  problem_data[pid]['llama_output'] = get_output(problem_data[pid]['image_captioned_prompt'])
  #print(problem_data[pid]['llama_output'])
  print()

In [37]:
# Specify the path to your JSON file
json_file_path = main_path + 'llava_cot_llama_combined_test_result.json'
problem_data = read_json(json_file_path)
last_id =0

In [ ]:
from llmengine import Completion

for pid, content in problem_data.items():
  if int(pid) >= int(last_id):
    print(pid)
    response = Completion.create(
      model="llama-2-7b",
      prompt=problem_data[pid]['image_captioned_prompt'],
      max_new_tokens=1,
      temperature=0.01,
    )
    print()
    print(response.output.text)
    problem_data[pid]['llama_zero_shot_output'] = response.output.text


In [40]:
last_id = '879'
print(problem_data['865']['llama_zero_shot_output'])

0


In [41]:
file_path = main_path+'llava_cot_llama_zeroshot_combined_test_result.json'

# Save the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(problem_data, json_file)